In [1]:
import requests
import json
import pandas as pd
import numpy as np
import csv
import os

In [3]:
url = "http://www.omdbapi.com/?t="
api_key = "&apikey=trilogy"
fp = os.path.join("Resources","MovieList.csv") #File Path to table of movies to lookup

In [4]:
# Breaks up string into just first names
def NameSplit(names):
    firstNames = []
    # splits list into names
    for name in names.split(", "):
        # if full name pulls out first name
        firstNames.append(name.split()[0])
    # If there was no name, returns nan
    if((len(firstNames)<=1) & (len(firstNames[0])<=3)):return(np.nan)
    return(firstNames)

In [5]:
# sets empty df
df = pd.DataFrame(columns = ['Title','Year','Actors','Characters'])

with open(fp,newline="")as csvf:
    csvr = csv.reader(csvf,delimiter=',')
    # Skips header
    next(csvr)
    for row in csvr:
        # initilizes movie name and then sets it.
        name = ""
        name = row[1]
        # response for movie pull
        res = requests.get(url+name+api_key).json()
        
        # calling function namesplit as it returns a list as one string
        # some movies dont have a character list
        try:
            char = NameSplit(row[4])
        except:
            # if it failed then set to nan
            char = np.nan
        # year movie came out
        year = row[0]
        # some movies dont have actor list
        try:
            actors = NameSplit(str(res['Actors']))
        except:
            actors = np.nan
        
        # appending new data to table
        df = df.append({"Title":name,'Year':year,'Actors':actors,'Characters':char},ignore_index=True)
        
# drop rows from table that failed to return both actor and characer names
df.dropna(thresh=3,inplace=True)
df.head()

,Title,Year,Actors,Characters
0,The Birth of a Nation,1915,"[Henry, Lillian, Mae, Miriam]",NaN
2,Cleopatra,1917,"[Elizabeth, Richard, Rex, Pamela]",[Cleopatra]
3,Mickey,1918,"[Mabel, George, Laura, Lew]",[Mickey]
4,The Miracle Man,1919,"[Lon, Betty, Joseph, J.M.]",NaN
5,Way Down East,1920,"[Lillian, Richard, Mrs., Lowell]",NaN


In [6]:
# this cell was for testing how many unique names were 
# pulled amungst both actors and characters
# does not need to be run for pulling
nameList=[]
for value in df['Actors']:
    try:
        for name in value:
            if name not in nameList: nameList.append(name)
    except:
        pass
for value in df['Characters']:
    try:
        for name in value:
            if name not in nameList: nameList.append(name)
    except:
        pass

In [7]:
# saves table out to csv
df.to_csv('Resources/MovieTable.csv',index=False)

In [8]:
# intilizes cleaned table
nameTable = pd.DataFrame(columns=['Name','Year','Movie'])

# creating table to join with Chris's Data Table
with open('Resources/MovieTable.csv',newline="") as csvf:
    csvr = csv.reader(csvf,delimiter=',')
    next(csvr)
    for row in csvr:
        # due to how it reads in data, have to clean up a bit
        for name in row[2].split(', '):
            name = name.replace('[',"")
            name = name.replace('\'',"")
            name = name.replace(']',"")
            nameTable = nameTable.append({'Name':name,'Year':row[1],
                              'Movie':row[0]},ignore_index=True)
        for name in row[3].split(', '):
            name = name.replace('[',"")
            name = name.replace('\'',"")
            name = name.replace(']',"")
            nameTable = nameTable.append({'Name':name,'Year':row[1],
                              'Movie':row[0]},ignore_index=True)
# removes "" rows from table
nameTable = nameTable.loc[nameTable['Name']!=""]
nameTable.head()

,Name,Year,Movie
0,Henry,1915,The Birth of a Nation
1,Lillian,1915,The Birth of a Nation
2,Mae,1915,The Birth of a Nation
3,Miriam,1915,The Birth of a Nation
5,Elizabeth,1917,Cleopatra


In [9]:
# saves out to table
nameTable.to_csv('Resources/NameTable.csv',index=False)